In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


In [2]:
# from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import * 

from torch.utils.tensorboard import SummaryWriter

# # Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd
from models import *

# # Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *

from transfer_attacks.Boundary_Transferer import *
from transfer_attacks.projected_gradient_descent import *

In [3]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedEM_adv"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 6
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_12_27_feddef1_n40/'
args_.validation = False

In [11]:
# Other Argument Parameters
Q = 2 # update per round
G = 0.5
num_clients = 40
S = 0.05 # Threshold
step_size = 0.01
K = 15

# Randomized Parameters
Ru = np.random.uniform(low=0.2, high=0.8, size=num_clients)

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_clients)

# Set attack parameters
x_min = torch.min(clients[0].adv_nn.dataloader.x_data)
x_max = torch.max(clients[0].adv_nn.dataloader.x_data)
atk_params = PGD_Params()
atk_params.set_params(batch_size=1, iteration = K,
                   target = -1, x_val_min = x_min, x_val_max = x_max,
                   step_size = 0.05, step_norm = "inf", eps = 1, eps_norm = "inf")

# Obtain the central controller decision making variables (static)
num_h = args_.n_learners= 3
Du = np.zeros(len(clients))

for i in range(len(clients)):
    num_data = clients[i].train_iterator.dataset.targets.shape[0]
    Du[i] = num_data
D = np.sum(Du) # Total number of data points


==> Clients initialization..
===> Building data iterators..



100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 212.90it/s]


===> Initializing clients..



100%|███████████████████████████████████████████| 80/80 [00:47<00:00,  1.69it/s]


==> Test Clients initialization..
===> Building data iterators..



0it [00:00, ?it/s]


===> Initializing clients..



0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.195% |Test Loss: 2.292 | Test Acc: 12.291% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [13]:
# Train the model
print("Training..")
pbar = tqdm(total=args_.n_rounds)
current_round = 0
while current_round <= args_.n_rounds:

    # If statement catching every Q rounds -- update dataset
    if  current_round%Q == 0: # current_round != 0 and
        print("Round:", current_round, "Calculation Adv")
        # Obtaining hypothesis information
        Whu = np.zeros([num_clients,num_h]) # Hypothesis weight for each user
        for i in range(len(clients)):
            # print("client", i)
            temp_client = aggregator.clients[i]
            hyp_weights = temp_client.learners_ensemble.learners_weights
            Whu[i] = hyp_weights

        row_sums = Whu.sum(axis=1)
        Whu = Whu / row_sums[:, np.newaxis]
        Wh = np.sum(Whu,axis=0)/num_clients

        # Solve for adversarial ratio at every client
        Fu = solve_proportions(G, num_clients, num_h, Du, Whu, S, Ru, step_size)

        # Assign proportion and attack params
        # Assign proportion and compute new dataset
        for i in range(len(clients)):
            aggregator.clients[i].set_adv_params(Fu[i], atk_params)
            aggregator.clients[i].update_advnn()
            aggregator.clients[i].assign_advdataset()

    aggregator.mix()

    if aggregator.c_round != current_round:
        pbar.update(1)
        current_round = aggregator.c_round

Training..



7it [17:39, 151.33s/it]                                   | 0/6 [00:00<?, ?it/s]

Round: 0 Calculation Adv


KeyboardInterrupt: 

In [14]:
Fu

array([0.34468773, 0.2133766 , 0.51      , 0.46144176, 0.34613134,
       0.51      , 0.51      , 0.51      , 0.51      , 0.46828686,
       0.42820397, 0.51      , 0.51      , 0.51      , 0.51      ,
       0.51      , 0.51      , 0.39600709, 0.51      , 0.51      ,
       0.51      , 0.45382132, 0.51      , 0.4492753 , 0.51      ,
       0.41949363, 0.2650161 , 0.37923982, 0.49749974, 0.27260785,
       0.46606647, 0.20425306, 0.51      , 0.51      , 0.51      ,
       0.46925899, 0.47293463, 0.51      , 0.51      , 0.51      ])

In [12]:
Ru

array([0.34468773, 0.2133766 , 0.59060121, 0.46144176, 0.34613134,
       0.6316171 , 0.59091473, 0.61940604, 0.5276325 , 0.46828686,
       0.42820397, 0.60665422, 0.76395935, 0.55469548, 0.7876177 ,
       0.62082463, 0.52615043, 0.39600709, 0.73623018, 0.53855584,
       0.62927148, 0.45382132, 0.6573649 , 0.4492753 , 0.78654296,
       0.41949363, 0.2650161 , 0.37923982, 0.49749974, 0.27260785,
       0.46606647, 0.20425306, 0.71174091, 0.52123709, 0.63291406,
       0.46925899, 0.47293463, 0.76532351, 0.63200149, 0.76032822])

In [17]:
Wh

array([0.33333333, 0.33333333, 0.33333333])

In [4]:
import numba


In [5]:
device = numba.cuda.get_current_device()

In [6]:
device

<weakproxy at 0x7f2d8749cf40 to Device at 0x7f2d88ec17f0>

In [7]:
device.reset()

In [8]:
device

<weakproxy at 0x7f2d8749cf40 to Device at 0x7f2d88ec17f0>

In [12]:
x = 5
y = 3

In [13]:
del x, y

In [15]:
x

NameError: name 'x' is not defined